# BIP's error function

The evaluation of the error metric by the BIP error function.

It starts from a dataframe with predictions **NOT yet summed by month**.


The input dataframe should **at least** contain the attributes:

| StoreID |  D_Month | NumberOfSales | _NumberOfSales |
| --------| -------- | --------------| -------------- |  
|    1000 |        1 |   8540.000000 |    9776.962748 |  
|    1000 |        1 |  10364.000000 |    6018.866667 |  
|    1000 |        2 |   4676.000000 |    6234.658815 |  
|    1000 |        3 |   7675.446488 |    6513.012222 |  
|    1000 |        5 |   6267.000000 |    7242.277146 | 

Where:
 
 - *NumberOfSales* are the **real** values
 - *_NumberOfSales* are the **predicted** values

### **WARNING:** No check is done on the months. They can be any arbitrary month
### **WARNING 2:** This function expect the dataset NOT yet summed by month

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *
from BIP import apply_BIP_submission_format

train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test_m12_64_Model_XGBoost_final.csv')

The customer provided us the following code

In [2]:
def BIP_error(result, regions):
    def regional_error(v):
        y_true = v["Target" ]
        y_pred = v["NumberOfSales" ]
        return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

    def global_error(region_sums):
        return np.mean(region_sums)

    score = global_error(pd.merge(result, regions, on="StoreID")[["Region" , "Target" , "NumberOfSales" ]].groupby("Region" ).apply(regional_error))
    return score

Let's proceed to integrate it with what's required to make it work with our workflow

In [3]:
# extract the regions associated to the stores by the train set
regions = train.copy()
regions = regions[['StoreID', 'Region']]
regions.drop_duplicates(inplace=True)
regions.head()

,StoreID,Region
0,1000,7
729,1001,0
1458,1002,3
2187,1003,7
2916,1004,2


In [4]:
test.head()

,StoreID,Date,IsHoliday,HasPromotions,NearestCompetitor,Region,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,...,p4,p5,p6,p7,p8,p9,p10,p11,p12,_NumberOfSales
0,1000,01/01/2018,0,0,326,7,8540,9643,17130,2770,...,-0.000163,-0.002012,0.002307,0.001049,-0.001226,0.000372,0.000095,0.000836,-0.000170,8007.3823
1,1000,02/01/2018,0,0,326,7,10364,9643,17130,2770,...,-0.000491,-0.002070,0.002708,-0.000669,-0.001426,-0.000215,-0.000540,0.000367,-0.000163,6596.4365
2,1000,03/01/2018,0,0,326,7,4676,9643,17130,2770,...,-0.000582,-0.002125,0.003129,-0.000895,-0.001529,-0.000557,0.000031,-0.000107,-0.000037,5140.5054
3,1000,05/01/2018,0,0,326,7,6267,9643,17130,2770,...,-0.000389,0.002572,-0.000628,-0.000278,-0.001221,0.000540,0.000499,-0.000087,-0.000236,6469.8920
4,1000,06/01/2018,0,0,326,7,5953,9643,17130,2770,...,-0.000154,0.000616,0.003896,0.000200,-0.001384,-0.000087,-0.000091,0.000085,0.000273,5815.5290


In [5]:
# let's apply the BIP submission format to our results on tests
result = apply_BIP_submission_format(test)

In [6]:
result.head()

,StoreID,Month,Target,NumberOfSales
0,1000,1,182917,195912.7838
1,1000,2,166161,172333.8114
2,1001,1,95745,89660.7154
3,1001,2,88423,79868.3372
4,1002,1,121995,129124.9452


In [7]:
# apply the BIP provided code

step_by_step_error = BIP_error(result, regions)

print("BIP total error: {}".format(step_by_step_error))

BIP total error: 0.04913953326583034


# The `get_BIP_error` function

In [8]:
def get_BIP_error(df):
    """
    It's based on the BIP provided formula.
    Given a dataframe with predictions, **NOT yet summed by month** it return the total error
    done on predictions.
    
    Required attributes in the dataframe:
        ['StoreID', 'D_Month', 'NumberOfSales', '_NumberOfSales']
    
    :param df: The data frame
    :return: Dataframe in the submit format.
    """
    required_attributes = ['StoreID', 'D_Month', 'NumberOfSales', '_NumberOfSales']

    # create a copy of the dataframe to do not change the provided one
    df = df.copy()
    
    # Remove useless columns and select all the ones required.
    # Implicit check that all the required columns are present.
    df = df[required_attributes]
    
    
    def get_regions():
        # starting from the train set, get a dataframe which link the stores to their region
        regions = pd.read_csv('./dataset/train.csv')
        regions = regions[['StoreID', 'Region']]
        regions.drop_duplicates(inplace=True)
        return regions
    
    # BIP provided code
    def BIP_error(result, regions):
        def regional_error(v):
            y_true = v["Target" ]
            y_pred = v["NumberOfSales" ]
            return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

        def global_error(region_sums):
            return np.mean(region_sums)

        score = global_error(pd.merge(result, regions, on="StoreID")[["Region" , "Target" , "NumberOfSales" ]].groupby("Region" ).apply(regional_error))
        return score
    
    result = apply_BIP_submission_format(df)
    regions = get_regions()
    
    BIP_total_error = BIP_error(result, regions)
    print("BIP total error: {}".format(BIP_total_error))
    
    return BIP_total_error
    
    

### Function test

In [9]:
test = pd.read_csv('./dataset/test_m12_64_Model_XGBoost_final.csv')

In [10]:
error_from_BIP = get_BIP_error(test)

BIP total error: 0.04913953326583034


In [11]:
# let's assert that the two errors are equal
np.testing.assert_almost_equal(step_by_step_error, error_from_BIP, decimal=10)

### Test of the BIP error function: case of predictions = 0

In [12]:
test_zero = pd.read_csv('./dataset/test_m12_64_Model_XGBoost_final.csv')
test_zero['_NumberOfSales'] = 0
test_zero.head()

,StoreID,Date,IsHoliday,HasPromotions,NearestCompetitor,Region,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,...,p4,p5,p6,p7,p8,p9,p10,p11,p12,_NumberOfSales
0,1000,01/01/2018,0,0,326,7,8540,9643,17130,2770,...,-0.000163,-0.002012,0.002307,0.001049,-0.001226,0.000372,0.000095,0.000836,-0.000170,0
1,1000,02/01/2018,0,0,326,7,10364,9643,17130,2770,...,-0.000491,-0.002070,0.002708,-0.000669,-0.001426,-0.000215,-0.000540,0.000367,-0.000163,0
2,1000,03/01/2018,0,0,326,7,4676,9643,17130,2770,...,-0.000582,-0.002125,0.003129,-0.000895,-0.001529,-0.000557,0.000031,-0.000107,-0.000037,0
3,1000,05/01/2018,0,0,326,7,6267,9643,17130,2770,...,-0.000389,0.002572,-0.000628,-0.000278,-0.001221,0.000540,0.000499,-0.000087,-0.000236,0
4,1000,06/01/2018,0,0,326,7,5953,9643,17130,2770,...,-0.000154,0.000616,0.003896,0.000200,-0.001384,-0.000087,-0.000091,0.000085,0.000273,0


In [13]:
# which is the erro in this case? --> expect: 1.00
get_BIP_error(test_zero)

BIP total error: 1.0


1.0

### Test of the BIP error function: case of predictions = predictions + n

Let's test what happen if number of sales are incremented of a number *n*.

In [14]:
n = 100
test_n = pd.read_csv('./dataset/test_m12_64_Model_XGBoost_final.csv')
test_n['_NumberOfSales'] = test_n['NumberOfSales'] + n
test_n.head()

,StoreID,Date,IsHoliday,HasPromotions,NearestCompetitor,Region,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,...,p4,p5,p6,p7,p8,p9,p10,p11,p12,_NumberOfSales
0,1000,01/01/2018,0,0,326,7,8540,9643,17130,2770,...,-0.000163,-0.002012,0.002307,0.001049,-0.001226,0.000372,0.000095,0.000836,-0.000170,8640
1,1000,02/01/2018,0,0,326,7,10364,9643,17130,2770,...,-0.000491,-0.002070,0.002708,-0.000669,-0.001426,-0.000215,-0.000540,0.000367,-0.000163,10464
2,1000,03/01/2018,0,0,326,7,4676,9643,17130,2770,...,-0.000582,-0.002125,0.003129,-0.000895,-0.001529,-0.000557,0.000031,-0.000107,-0.000037,4776
3,1000,05/01/2018,0,0,326,7,6267,9643,17130,2770,...,-0.000389,0.002572,-0.000628,-0.000278,-0.001221,0.000540,0.000499,-0.000087,-0.000236,6367
4,1000,06/01/2018,0,0,326,7,5953,9643,17130,2770,...,-0.000154,0.000616,0.003896,0.000200,-0.001384,-0.000087,-0.000091,0.000085,0.000273,6053


In [15]:
# which is the erro in this case?
get_BIP_error(test_n)

BIP total error: 0.021475322132617925


0.021475322132617925